In [ ]:
# |default_exp mongo_livre

# Livre

In [ ]:
# |export

from mongo import BaseEntity, Editeur, get_collection
from config import get_DB_VARS
from mongo_auteur import Auteur
from typing import List, Type, TypeVar
from bson import ObjectId


T = TypeVar("T", bound="Livre")


class Livre(BaseEntity):
    collection: str = "livres"

    def __init__(self, titre: str) -> None:
        """Initialise une instance de livre.

        Args:
            nom (str): Le titre du livre.
        """
        super().__init__(titre, self.collection)
        self.titre = titre  # je le duplique pour la comprehension du concept de livre
        self.auteur = None  # on mettra l'oid de l'auteur
        self.editeur = None  # on mettra l'oid de l'editeur

    def add_auteur(self, auteur: Auteur):
        if auteur is not None:
            self.auteur = auteur.get_oid()

    def add_editeur(self, editeur: Editeur):
        if editeur is not None:
            self.editeur = editeur.get_oid()

    @classmethod
    def with_details(cls, titre: str, auteur: Auteur, editeur: Editeur):
        """Alternative constructor to instantiate a Livre with title, auteur and editeur.

        Args:
            titre (str): The title of the book.
            auteur (Auteur): Instance of Auteur.
            editeur (Editeur): Instance of Editeur.
        Returns:
            Livre: An instance of Livre.
        """
        instance = cls(titre)
        instance.add_auteur(auteur)
        instance.add_editeur(editeur)
        return instance

    @classmethod
    def from_oid(cls: Type[T], oid: ObjectId) -> T:
        """Creates an instance of Livre class from a MongoDB ObjectId.
        Returns None if the ObjectId is not found in the database or is None.

        Args:
            oid (ObjectId): The MongoDB ObjectId.

        Returns:
            T: An instance of the derived class.
        """
        if oid is None:
            return None
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=cls.collection
        )
        document = collection.find_one({"_id": oid})
        if document is None:
            return None
        inst = cls.with_details(
            document.get("titre"),
            Auteur.from_oid(document.get("auteur")),
            Editeur.from_oid(document.get("editeur")),
        )
        return inst

    def __str__(self) -> str:
        """Official string representation of the entity.

        Returns:
            str: The name of the entity: Titre, Auteur, Editeur.
        """
        return f"""
        Titre: {self.titre}
        Auteur: {Auteur.from_oid(self.auteur) if self.auteur is not None else None}
        Editeur: {Editeur.from_oid(self.editeur) if self.editeur is not None else None}
        """

In [ ]:
ez = Auteur("Émile Zola")
assert ez.exists()

poche = Editeur("Le Livre de Poche")
poche.keep()
assert poche.exists()

l = Livre("Germinal")
l.add_auteur(ez)
l.add_editeur(poche)
l.keep()
print(l)


        Titre: Germinal
        Auteur: Émile Zola
        Editeur: Le Livre de Poche
        


In [ ]:
s = Auteur("Stendhal")
assert s.exists()

l = Livre("Le Rouge et le Noir")
l.add_auteur(s)
l.keep()
print(l)


        Titre: Le Rouge et le Noir
        Auteur: Stendhal
        Editeur: None
        


# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("py mongo helper livres.ipynb", ".")